<center><h1>chen_jacqueline_HW7</h1></center>
<br>
<br>


Name: Jacqueline Chen
<br>
Github Username: jchen741
<br>
USC ID: 7983593463

USING ONE LATE DAY

## 1. Multi-class and Multi-Label Classification Using Support Vector Machines

Import packages

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as imbpipe
from sklearn.metrics import hamming_loss
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

### (a) Download the Anuran Calls (MFCCs) Data Set

In [4]:
data=pd.read_csv('../data/Anuran Calls (MFCCs)/Frogs_MFCCs.csv')
X_train,X_test,y_train,y_test=train_test_split(data.iloc[:,:-4],data[['Family','Genus','Species']],random_state=42,test_size=0.3)

### (b) Train a classifier for each label

#### (i) Research

Exact Match Ratio (EMR): measures percentage of instances where the classifier predicts correctly, exactly matching the true labels.

Hamming Loss/Score: fraction of labels that are incorrectly predicted, averaged over all the instances. hamming score is the complement of hamming loss. hamming score calculates the mismatched distance between the predicted and true value and then sums it up as an average

#### (ii) Train a SVM for each of the labels

In [5]:
def hamming_loss_(y_test,y_pred):
    outputs = []
    cols = y_test.columns
    for col in cols:
        outputs.append(hamming_loss(y_test[col],y_pred[col]))
    return np.mean(outputs)

In [6]:
def EMR(y_test,y_pred):
    y_pred = pd.DataFrame.from_dict(y_pred)
    return np.all(y_pred.to_numpy() == y_test.to_numpy(),axis=1).mean()        

In [7]:
y_pred=dict()
cols=['Family','Genus','Species']

for col in cols:
    svc=SVC(kernel='rbf',decision_function_shape='ovr')
    paramgrid={'C':[10**i for i in range (-3,3)],'gamma':[i/10 for i in range(1,20)]}
    stratified_cv=StratifiedKFold(n_splits = 10)
    clf=GridSearchCV(svc, param_grid=paramgrid,cv=stratified_cv)
    clf.fit(X_train,y_train[col])
    print('label=', col, 'best score=', clf.best_score_, 'best params=', clf.best_params_)
    y_pred[col]=clf.predict(X_test)
print('EMR=', EMR(y_test,y_pred))
print('hamming loss=', hamming_loss_(y_test,y_pred))


label= Family best score= 0.9932496292088737 best params= {'C': 10, 'gamma': 1.9}
label= Genus best score= 0.9908647322414719 best params= {'C': 100, 'gamma': 1.5}
label= Species best score= 0.9904675123860013 best params= {'C': 10, 'gamma': 1.8}
EMR= 0.9874942102825383
hamming loss= 0.008337193144974525


#### (iii) Repeat 1(b)ii with L1-penalized SVMs

In [8]:
y_pred=dict()
# using linear per convention mentioned in hw assignment footnotes

for col in cols:
    linear_svc=LinearSVC(penalty='l1',loss='squared_hinge',dual=False,max_iter=1000000)
    pipe=Pipeline([('scale',StandardScaler()),('linear',linear_svc)])
    paramgrid={'linear__C':[10**i for i in range (-3,3)]}
    stratified_cv=StratifiedKFold(n_splits = 10)
    clf=GridSearchCV(pipe, param_grid=paramgrid,cv=stratified_cv)
    clf.fit(X_train,y_train[col])
    print('label=', col, 'best score=', clf.best_score_, 'best params=', clf.best_params_)
    y_pred[col]=clf.predict(X_test)
print('EMR=', EMR(y_test,y_pred))
print('hamming loss=', hamming_loss_(y_test,y_pred))

label= Family best score= 0.9406288459717882 best params= {'linear__C': 1}
label= Genus best score= 0.9527422764997319 best params= {'linear__C': 10}
label= Species best score= 0.9602847202499291 best params= {'linear__C': 10}
EMR= 0.9129226493747105
hamming loss= 0.05681642735834491


#### (iv) Repeat 1(b)iii by using SMOTE or any other method for imbalance

In [9]:
y_pred=dict()
# using smote to combat class imbalance

for col in cols:
    linear_svc=LinearSVC(penalty='l1',loss='squared_hinge',dual=False,max_iter=1000000)
    pipe=imbpipe([('smote',SMOTE()),('scale',StandardScaler()),('linear',linear_svc)])
    paramgrid={'linear__C':[10**i for i in range (-3,3)]}
    stratified_cv=StratifiedKFold(n_splits = 10)
    clf=GridSearchCV(pipe, param_grid=paramgrid,cv=stratified_cv)
    clf.fit(X_train,y_train[col])
    print('label=', col, 'best score=', clf.best_score_, 'best params=', clf.best_params_)
    y_pred[col]=clf.predict(X_test)
print('EMR=', EMR(y_test,y_pred))
print('hamming loss=', hamming_loss_(y_test,y_pred))

label= Family best score= 0.9219662185616461 best params= {'linear__C': 1}
label= Genus best score= 0.9142214175265865 best params= {'linear__C': 100}
label= Species best score= 0.9569081542491086 best params= {'linear__C': 10}
EMR= 0.8578045391384901
hamming loss= 0.07534352323606608


Results: Gaussian SVM from a is better than L1 penalized, which is better than SMOTE for both the EMR and hamming loss evaluation metrics

## 2. K-Means Clustering on a Multi-Class and Multi-Label Data Set

### (a) Use k-means clustering

In [10]:
df=data.iloc[:,:-1]
X, y=df.iloc[:,:-3],data[cols]

def cluster_hamming_loss(y_test,y_pred):
    outputs=[]
    cols=y_pred.keys()
    for col in cols:
        outputs.append(hamming_loss([y_test[col]],[y_pred[col]]))
    return np.mean(outputs)
def cluster_hamming_distance(y_test,y_pred):
    outputs=[]
    cols=y_pred.keys()
    for col in cols:
        outputs.append(hamming_loss([y_test[col]],[y_pred[col]]))
    return np.sum(outputs)
def EMR_(y_test,y_pred):
    y_pred=pd.DataFrame.from_dict(y_pred)
    return np.all(y_pred.to_numpy() == y_test.to_numpy(),axis=1).mean()
def my_best_k():
    clusters=range(2,5)
    best_k,best_score = 1,0
    for cluster in clusters:
        clusterer=KMeans(n_clusters=cluster,init='random')
        pred=clusterer.fit_predict(X)
        center=clusterer.cluster_centers_
        score=silhouette_score(X,pred)
        if score>best_score:
            best_score = score
            best_k = cluster
    return best_k,best_score

### (b) Determine which family is the majority

### (c) Calculate the average Hamming distance, Hamming score, and Hamming loss

In [11]:
monte_hamming=[]
import warnings
warnings.filterwarnings("ignore", category=FutureWarning, module="sklearn")

for i in range(0,50):
    warnings.filterwarnings("ignore", category=FutureWarning, module="sklearn")

    print(i,':')
    best_k, best_score=my_best_k()
    
    print('best k=',best_k,' with score of',best_score)
    clusterer=KMeans(n_clusters=best_k)
    pred=clusterer.fit_predict(X)
    center=clusterer.cluster_centers_
    pred_labels=dict()

    for k in range(best_k):
        pred_labels[k]=dict()
        for col in cols:
            pred_labels[k][col]=dict()

    for i in range(len(pred)):
        k=pred[i]
        for col in cols:
            true=data.iloc[i][col]
            pred_labels[k][col][true]=pred_labels[k][col].get(true,0)+1
    majority_labels=dict()
    for k in pred_labels:
        majority_labels[k]=dict()
        for col in cols:
            majority,major_class=0,'wrong'

            for key in pred_labels[k][col]:
                val=pred_labels[k][col][key]
                if val>majority:
                    majority=val
                    major_class=key

            majority_labels[k][col]=major_class

    print('majority labels=',majority_labels)
    hamming_loss_arr=[]
    hamming_distance_arr=[]
    for i in range(len(data)):
        hamming_loss_arr.append(cluster_hamming_loss(data[cols].iloc[i],majority_labels[pred[i]]))
        hamming_distance_arr.append(cluster_hamming_distance(data[cols].iloc[i],majority_labels[pred[i]]))
    
    avg_hamming_loss=np.mean(hamming_loss_arr)
    avg_hamming_distance=np.mean(hamming_distance_arr)
    print('avg hamming loss=',avg_hamming_loss)
    print('avg hamming distance=',avg_hamming_distance)
    print('avg hamming score',1-avg_hamming_loss)
    monte_hamming.append(avg_hamming_distance)

0 :
best k= 4  with score of 0.3787509343305295
majority labels= {0: {'Family': 'Hylidae', 'Genus': 'Hypsiboas', 'Species': 'HypsiboasCordobae'}, 1: {'Family': 'Leptodactylidae', 'Genus': 'Adenomera', 'Species': 'AdenomeraHylaedactylus'}, 2: {'Family': 'Dendrobatidae', 'Genus': 'Ameerega', 'Species': 'Ameeregatrivittata'}, 3: {'Family': 'Hylidae', 'Genus': 'Hypsiboas', 'Species': 'HypsiboasCinerascens'}}
avg hamming loss= 0.22233032198285846
avg hamming distance= 0.6669909659485754
avg hamming score 0.7776696780171415
1 :
best k= 4  with score of 0.3787509343305295
majority labels= {0: {'Family': 'Leptodactylidae', 'Genus': 'Adenomera', 'Species': 'AdenomeraHylaedactylus'}, 1: {'Family': 'Hylidae', 'Genus': 'Hypsiboas', 'Species': 'HypsiboasCordobae'}, 2: {'Family': 'Hylidae', 'Genus': 'Hypsiboas', 'Species': 'HypsiboasCinerascens'}, 3: {'Family': 'Dendrobatidae', 'Genus': 'Ameerega', 'Species': 'Ameeregatrivittata'}}
avg hamming loss= 0.22242297892054666
avg hamming distance= 0.667268

In [12]:
print('avg of hamming distance=',np.mean(monte_hamming))
print('sd of hamming distance =',np.std(monte_hamming))

avg of hamming distance= 0.6783349548297426
sd of hamming distance = 0.04482047186535554


## 3. ISLR 12.6.2

![Alt text](image.png)